# Export settings

When exporting to html, get plotly.js from CDN rather than including in the html export. Saves some space. Also use dark theme.

In [1]:
import plotly.io as pio
pio.renderers.default = "iframe"
pio.templates.default  = "plotly_dark"

# Imports

Import some libraries

In [2]:
import numpy as np
from sympy import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# The polar_plot function

Below we define the polar_plot function. Feel free to take a look - currently it lacks any kind of documentation though!

In [3]:
animation_opts = dict(frame=dict(duration=10, redraw=False),                                                
                         transition=dict(duration=0),
                         easing='linear',
                         fromcurrent=True,
                         mode='immediate')
def get_range(a):
    amin = np.min(a)
    amax = np.max(a)
    pad = 0.05 * (amax - amin)
    return [amin - pad, amax + pad]

def polar_plot(f):
    theta_vals = np.linspace(0, 2*np.pi, 1000)
    
    if f.is_constant():
        r_vals = np.full_like(theta_vals, N(f))
    else:
        flambda = lambdify(tuple(f.free_symbols), f)
        r_vals = flambda(theta_vals)
    
    x_vals = r_vals * np.cos(theta_vals)
    y_vals = r_vals * np.sin(theta_vals)

    xy_title =  r"$(x, y) = [" + latex(f) + r"]\, * \,(\cos\theta, \sin\theta)$"
    rtheta_title =  r"$r = " + latex(f) + r"$"
    fig = make_subplots(rows=2, cols = 1, subplot_titles = (xy_title, rtheta_title), row_heights=[0.8, 0.2], vertical_spacing = 0.15)
    
    fig.update_xaxes(range = get_range(x_vals), showticklabels=False, row=1, col = 1)
    fig.update_yaxes(range = get_range(y_vals), showticklabels=False, scaleanchor = "x1", scaleratio = 1, row=1, col=1)
    fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode="lines", line={"color" : "blue"}), row=1, col=1)
    fig.add_trace(go.Scatter(x=[x_vals[0]], y=[y_vals[0]], mode="markers", marker={"color" : "red", "size" : 10}), row=1, col=1)    
    fig.add_trace(go.Scatter(x=[0, x_vals[0]], y = [0, y_vals[0]], mode="lines", line={"color": "red"}), row=1, col=1)
    
    fig.update_xaxes(range = get_range(theta_vals), showticklabels=False, row=2, col = 1)
    fig.update_yaxes(range = get_range(r_vals), showticklabels=False, scaleanchor = "x2", scaleratio = 1, row=2, col=1)
    fig.add_trace(go.Scatter(x=theta_vals, y=r_vals, mode="lines", line={"color" : "red"}), row=2, col=1)
    fig.add_trace(go.Scatter(x=[theta_vals[0]], y=[r_vals[0]], mode="markers", marker={"color" : "red", "size" : 10}), row=2, col=1)
    
    fig.update_layout(showlegend=False, width=400, height=600)
        
    fig.layout.annotations[0]["y"] += 0.05
    fig.layout.annotations[1]["y"] += 0.05
    
    L = len(theta_vals)
    frames = [go.Frame(name = k, 
                       data = [go.Scatter(x=[x_vals[k]], y=[y_vals[k]]),
                               go.Scatter(x=[0, x_vals[k]], y=[0, y_vals[k]]),
                               go.Scatter(x=[theta_vals[k]], y=[r_vals[k]])], 
                       traces = [1, 2, 4]) for k in range(L)]
    fig.update(frames = frames)
    
    buttons = [dict(label="Play", method="animate", args=[None, animation_opts])]
    menus = [dict(type="buttons", buttons=buttons)]                                                
    fig.update_layout(updatemenus = menus)
    
    return fig

def show(fig):
    fig.show(auto_play=True, include_plotlyjs = "cdn", animation_opts=animation_opts)

# Create a symbolic variable

Our functions will be symbolic expressions. These are defined using sympy. Here we create a symbol for the independent variable of our radial function \(r = f(\theta)\). Then we can use any sympy expression involving \(\theta\) to build our functions.

In [4]:
theta = symbols('\\theta')

# Constant Function

$f(\theta) = 1$

In [5]:
f = simplify(1)
fig = polar_plot(f)
show(fig)

# Limaçon

$f(\theta) = 1.3 + \cos(\theta)$

In [6]:
f = 1.3 + cos(theta)
fig = polar_plot(f)
show(fig)

# Cardiod

$f(\theta) = 1 - \cos(\theta)$

In [7]:
f = 1 - cos(theta)
fig = polar_plot(f)
show(fig)

# Two loop Limaçon

$f(\theta) = 1 + 3 \cos(\theta)$

In [8]:
f = 1 + 3*cos(theta)
fig = polar_plot(f)
show(fig)

# Spiral

$f(\theta) = \theta$

In [9]:
f = theta
fig = polar_plot(f)
show(fig)

# Multi-leaved flower

$f(\theta) = \sin(2\theta) \cos(2\theta)$

In [10]:
f = sin(2*theta)*cos(2*theta)
fig = polar_plot(f)
show(fig)

# Cartoon Face

$f(\theta) = \sin(2^{\theta}) - 1.7$

Function definition from https://www.intmath.com/plane-analytic-geometry/8-curves-polar-coordinates.php

In [11]:
f = sin(2**theta) - 1.7
fig = polar_plot(f)
show(fig)